In [1]:
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


#mysql database details
connection = mysql.connector.connect(host = 'localhost', user = 'root', password = 'chennai', database = 'tourism',auth_plugin='mysql_native_password')
mycursor = connection.cursor()

In [3]:
sql = """
SELECT t.TransactionId, u.UserId, t.visityear, t.visitmonth, t.attractionid, t.rating, u.contenentid, u.regionid, u.countryid, u.cityid, r.region, c.country, m.visitmode,
i.attractioncityid, i.attractiontypeid, i.attraction, i.attractionaddress, ct.contenent, cty.cityname, tp.attractiontype
FROM transaction t
INNER JOIN user u
ON t.userID = u.userID
inner join region r
on u.regionid = r.regionid
inner join country c
on c.countryid = u.countryid
inner join mode m
on m.visitmodeid = t.visitmode
inner join item i
on i.attractionid = t.attractionid
inner join continent ct
on ct.contenentid = u.contenentid
inner join city cty
on u.cityid = cty.cityid
inner join type tp
on tp.attractiontypeid = i.attractiontypeid

"""


mycursor.execute(sql)
data = mycursor.fetchall()
column_names = [data[0] for data in mycursor.description]


df = pd.DataFrame(data, columns = column_names)
df.head()

,TransactionId,UserId,visityear,visitmonth,attractionid,rating,contenentid,regionid,countryid,cityid,region,country,visitmode,attractioncityid,attractiontypeid,attraction,attractionaddress,contenent,cityname,attractiontype
0,0000051502,0000039119,2014,7,000824,4,01,02,0005,000011,East Africa,Kenya,Solo,01,076,Uluwatu Temple,"Jl. Raya Uluwatu Southern part of Bali, Pecatu...",Africa,Nairobi,Religious Sites
1,0000100364,0000008505,2017,10,000749,4,01,02,0011,000040,East Africa,Zambia,Couples,01,093,Tegenungan Waterfall,"Jl. Raya Tegenungan, Kemenuh, Ubud 80581 Indon...",Africa,Solwezi,Waterfalls
2,0000071692,0000071291,2016,9,000748,4,01,03,0014,000052,North Africa,Egypt,Couples,01,072,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Africa,Cairo,Points of Interest & Landmarks
3,0000073822,0000021388,2015,5,000748,5,01,03,0014,000052,North Africa,Egypt,Couples,01,072,Tegalalang Rice Terrace,"Jalan Raya Ceking, Tegalalang 80517 Indonesia",Africa,Cairo,Points of Interest & Landmarks
4,0000050237,0000058774,2015,9,000824,4,01,03,0014,000052,North Africa,Egypt,Couples,01,076,Uluwatu Temple,"Jl. Raya Uluwatu Southern part of Bali, Pecatu...",Africa,Cairo,Religious Sites


In [5]:
user_recom = df
user_item_matrix = user_recom.pivot_table('rating', ['UserId'], 'attractionid')
user_item_matrix.fillna(0, inplace=True)
user_mat_sim = cosine_similarity(user_item_matrix)

In [7]:
def collabfill(user_id, user_mat_sim, user_item_matrix, user_recom):
    num_users = user_mat_sim.shape[0]

    if user_id < 1 or user_id > num_users:
        print(f"Error: User ID {user_id} is out of range. Valid range: 1 to {num_users}")
        return pd.DataFrame()

    simuser = user_mat_sim[user_id - 1]
    sim_user_ids = np.argsort(simuser)[::-1][1:6]
    sim_user_rating = user_item_matrix.iloc[sim_user_ids].mean(axis=0)
    rec_dest_id = sim_user_rating.sort_values(ascending=False).head(10).index
    rec = user_recom[user_recom['attractionid'].isin(rec_dest_id)][['attraction', 'visitmode', 'rating']].drop_duplicates().head(10)

    return rec


In [9]:
col = collabfill(200, user_mat_sim,user_item_matrix, user_recom)
col

,attraction,visitmode,rating
7,Nusa Dua Beach,Family,4
14,Kuta Beach - Bali,Couples,5
44,Kuta Beach - Bali,Family,3
53,Kuta Beach - Bali,Family,5
64,Kuta Beach - Bali,Couples,3
65,Kuta Beach - Bali,Couples,1
72,Sewu Temple,Couples,4
133,Merapi Volcano,Couples,5
134,Merapi Volcano,Couples,3
137,Malioboro Road,Solo,4
